In [1]:
from multiprocessing import Process

In [7]:
def funcao(name):
    print("hello, sou ", name)

In [29]:
p = Process(target=funcao, args=("bob filho", )) # tem que ter essa virgula e vazio no fim pra dizer que acabou. 
# dá um erro bizarro se não fizer
p.start()
print("hello, sou ", "bob pai")
funcao("eu")
p.join()

hello, sou  bob filho
hello, sou  bob pai
hello, sou  eu


In [40]:
# Fazer em 4x paralelo
def funcao2(name, i):
    print("hello, sou ", name, i)

procs = []
for i in range(4):
    p = Process(target=funcao2, args=("bob filho", i,  ))
    procs.append(p)

print("hello, sou ", "bob pai")

for i in range(4):
    procs[i].start()
for i in range(4):
    procs[i].join() # o join tem que estar em for separado pra ele poder juntar tudo depois de executar.
    # se não executaria um por vez, não sendo paralelo

# dá um negócio louco pq tá executando os prints fora de ordem

hello, sou  bob pai
hello, sou hello, sou   hello, sou hello, sou bob filhobob filho  3   1bob filho
bob filho2
 0



In [41]:
#calculo de pi sequencial
import time
def pi_naive(start, end, step): # calcula pi somando área de retângulos
    print("Start: ", str(start))
    print("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    return soma

num_steps = 10000000
sums = 0.0
step = 1.0/num_steps
tic = time.time() #tempo inicial
sums = pi_naive(0, num_steps, step)
toc = time.time() #tempo final
pi = step*sums
print("Valor pi: %.10f" %pi)
print("Tempo pi: %.8f s" %(toc-tic)) # calcula tempo de execução

Start:  0
End:  10000000
Valor pi: 3.1415926536
Tempo pi: 0.90675044 s


In [71]:
#calculo de pi paralelo (feito pelo prof)
from multiprocessing import Process
import time
PROCS = 2
def pi(start, end, step):
    print ("Start: ", str(start))
    print ("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        sum = sum + 4.0/(1.0+x*x)
    print(sum)

num_steps = 100000000 #100.000.000
sum = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
workers = []

for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, ))
    workers.append(worker)
for worker in workers :
    worker.start()
for worker in workers :
    worker.join()
toc = time.time()
print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
Start:  50000000
End:  99999999
End:  49999999
128700219.75867517
185459040.40031576
Tempo Pi: 4.70304894 s


In [74]:
#calculo de pi paralelo (diferente do prof) isso aqui é 10x mais difícil que usar pipes
#, que é a solução do professor logo abaixo
import multiprocessing


def pi_naive(start, end, step, proc_num, return_list): # calcula pi somando área de retângulos
    print("Start: ", str(start))
    print("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    return_list.append(soma)


if __name__ == "__main__":
    start = 0
    end = 100000000 # 100.000.000
    sums = 0.0
    step = 1.0/end
    manager = multiprocessing.Manager()
    return_list = manager.list()
    jobs = []
    tic = time.time()
    for proc_num, start, end in zip([0, 1], [0, end//2], [end//2, end]):
        p = multiprocessing.Process(target=pi_naive, args=(start, end, step  ,proc_num, return_list))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    toc = time.time()
    print ("Tempo Pi: %.8f s" %(toc-tic))
    print(return_list[0] + return_list[1])

Start:  0Start: 
End:  50000000 
50000000End: 
 100000000
Tempo Pi: 4.75239038 s
314159265.35899097


In [85]:
# paralelo usando pipe
from multiprocessing import Process, Pipe
import time
PROCS = 2
def pi(start, end, step, entrada):
    print ("Start: ", str(start))
    print ("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        sum = sum + 4.0/(1.0+x*x)
    print(sum)
    entrada.send(sum)
    
    
entrada , saida = Pipe()
num_steps = 100000000 #100.000.000
sum = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
workers = []

worker = Process()
for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, entrada, ))
    workers.append(worker)
for worker in workers :
    worker.start()
lista = []
while len(lista) < PROCS:
    lista.append(saida.recv())
print((lista[0]+lista[1]))

for worker in workers :
    worker.join()
toc = time.time()
print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
Start: End:  49999999
 50000000
End:  99999999
185459040.40031576
128700219.75867517
314159260.1589909
Tempo Pi: 4.71264911 s
